FPL

In [1]:
import sys
from pathlib import Path

ROOT = Path.cwd().resolve()   # 보통 FPL 폴더에서 노트북 실행중이면 이게 루트
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))


import os, sys
from pathlib import Path
import cv2
import numpy as np
import pandas as pd

# 현재 노트북이 있는 폴더(FPL) 기준으로 src 절대경로 계산
SRC_DIR = (Path(os.getcwd()) / "src").resolve()

print("CWD:", os.getcwd())
print("SRC_DIR:", SRC_DIR)
print("SRC exists?:", SRC_DIR.exists())

sys.path.insert(0, str(SRC_DIR))
print("sys.path[0]:", sys.path[0])

from src.fpl_data_io import build_image_index, load_dataset
from src.fpl_features import extract_color_hs_3x3, extract_hog_3x3
from src.fpl_models import encode_road_labels, train_color_svm, train_hog_pca_svm_by_dims
from src.fpl_fusion import fuse_probabilities
from src.fpl_metrics import eval_accuracy, eval_confusion, eval_report
from src.fpl_detail_models import train_and_save_detail_models
from src.fpl_knn_models import train_and_save_knn_models

from src.fpl_features import (
    extract_color_hs_full,
    extract_hog_full,
)

from src.fpl_models import (
    encode_road_labels,
    train_color_svm,
    train_hog_pca_svm_by_dims,
    eval_svm,
    fit_sigmoid_calibrator,
    predict_proba_custom,
    fuse_probabilities,
    evaluate_fusion,
)

print("imports OK")


CWD: /home/hanseong/vscode/ML_code/FPL/FPL
SRC_DIR: /home/hanseong/vscode/ML_code/FPL/FPL/src
SRC exists?: True
sys.path[0]: /home/hanseong/vscode/ML_code/FPL/FPL/src
imports OK


In [2]:
# === PATHS ===
TRAINING_LABEL_CSV = "/home/hanseong/gdrive/ML_FPL_training_data/training_labels_plus.csv"
TEST_LABEL_CSV  = "/home/hanseong/gdrive/ML_FPL_test_data/test_labels_plus.csv"

test_path = "/home/hanseong/gdrive/ML_FPL_test_data"
training_path = "/home/hanseong/gdrive/ML_FPL_training_data"



In [3]:
# training/test 폴더 둘 다 훑어서 filename -> fullpath 인덱스 생성
image_index = build_image_index(training_path, test_path)

print("indexed files:", len(image_index))
print("sample:", list(image_index.items())[:3])


indexed files: 2974
sample: [('donhwamunro_11_da_A_raw_0824.jpg', '/home/hanseong/gdrive/ML_FPL_training_data/jpg/donhwamunro_11_da_A_raw_0824.jpg'), ('donhwamunro_11_da_A_raw_0825.jpg', '/home/hanseong/gdrive/ML_FPL_training_data/jpg/donhwamunro_11_da_A_raw_0825.jpg'), ('donhwamunro_11_da_A_raw_0826.jpg', '/home/hanseong/gdrive/ML_FPL_training_data/jpg/donhwamunro_11_da_A_raw_0826.jpg')]


In [4]:
train = load_dataset(TRAINING_LABEL_CSV, image_index, resize=True)
test  = load_dataset(TEST_LABEL_CSV, image_index, resize=True)

Training_origin_data = train["images"]
Test_data = test["images"]

training_road_label = train["road_labels"]
test_road_label = test["road_labels"]

training_photo_id = train["photo_ids"]
test_photo_id = test["photo_ids"]

training_detail = train["details"]
test_detail = test["details"]

training_filename = train["filenames"]
test_filename = test["filenames"]

print("Loaded train images:", len(Training_origin_data), "missed:", len(train["missed"]))
print("Loaded test  images:", len(Test_data), "missed:", len(test["missed"]))

print("Train sample image shapes:", {Training_origin_data[i].shape for i in range(min(5, len(Training_origin_data)))})
print("Test  sample image shapes:", {Test_data[i].shape for i in range(min(5, len(Test_data)))})


Loaded train images: 2231 missed: 0
Loaded test  images: 743 missed: 0
Train sample image shapes: {(682, 1024, 3)}
Test  sample image shapes: {(682, 1024, 3)}


In [5]:
training_x = train["xs"]; training_y = train["ys"]
test_x     = test["xs"];  test_y     = test["ys"]


In [6]:
y_train_road, y_test_road, road_label_map = encode_road_labels(training_road_label, test_road_label)

print("num roads:", len(road_label_map))
print("y_train_road unique:", np.unique(y_train_road))
print("y_test_road  unique:", np.unique(y_test_road))

# 매핑 일부 확인
items = list(road_label_map.items())
print("road_label_map sample:", items[:20])


num roads: 12
y_train_road unique: [ 0  1  2  3  4  5  6  7  8  9 10 11]
y_test_road  unique: [ 0  1  2  3  4  5  6  7  8  9 10 11]
road_label_map sample: [('donhwamunro', 0), ('donhwamunro_11', 1), ('donhwamunro_11_da', 2), ('donhwamunro_11_ga', 3), ('donhwamunro_11_na', 4), ('samildaero', 5), ('samildaero_26', 6), ('samildaero_28', 7), ('samildaero_30', 8), ('samildaero_32', 9), ('samildaero_32_ga', 10), ('suporo_28', 11)]


In [7]:

import importlib

# =========================
# 1) 모듈 import (as 별칭)
# =========================
import src.fpl_data_io as fpl_data_io
import src.fpl_features as fpl_features
import src.fpl_models as fpl_models
import src.fpl_fusion as fpl_fusion
import src.fpl_metrics as fpl_metrics
import src.fpl_detail_models as fpl_detail_models
import src.fpl_knn_models as fpl_knn_models

# =========================
# 2) reload
# =========================
importlib.reload(fpl_data_io)
importlib.reload(fpl_features)
importlib.reload(fpl_models)
importlib.reload(fpl_fusion)
importlib.reload(fpl_metrics)
importlib.reload(fpl_detail_models)
importlib.reload(fpl_knn_models)

# =========================
# 3) 함수 재바인딩 (⭐ 중요)
# =========================
# data io
build_image_index = fpl_data_io.build_image_index
load_dataset = fpl_data_io.load_dataset

# features
extract_color_hs_3x3 = fpl_features.extract_color_hs_3x3
extract_hog_3x3      = fpl_features.extract_hog_3x3
extract_color_hs_full = fpl_features.extract_color_hs_full
extract_hog_full      = fpl_features.extract_hog_full

# models
encode_road_labels = fpl_models.encode_road_labels
train_color_svm = fpl_models.train_color_svm
train_hog_pca_svm_by_dims = fpl_models.train_hog_pca_svm_by_dims
eval_svm = fpl_models.eval_svm
fit_sigmoid_calibrator = fpl_models.fit_sigmoid_calibrator
predict_proba_custom = fpl_models.predict_proba_custom

# fusion
fuse_probabilities = fpl_fusion.fuse_probabilities


# metrics
eval_accuracy = fpl_metrics.eval_accuracy
eval_confusion = fpl_metrics.eval_confusion
eval_report = fpl_metrics.eval_report

# detail / knn
train_and_save_detail_models = fpl_detail_models.train_and_save_detail_models
train_and_save_knn_models = fpl_knn_models.train_and_save_knn_models

print("✅ All src modules reloaded and rebound")


✅ All src modules reloaded and rebound


In [8]:
# HOG
X_hog_train = extract_hog_3x3(
    Training_origin_data,
    hog_size=(128, 128),
    orientations=12,
    pixels_per_cell=(8, 8),
    cells_per_block=(2, 2)
)
X_hog_test = extract_hog_3x3(
    Test_data,
    hog_size=(128, 128),
    orientations=12,
    pixels_per_cell=(8, 8),
    cells_per_block=(2, 2)
)

print("X_hog_train:", X_hog_train.shape)
print("X_hog_test :", X_hog_test.shape)


X_hog_train: (2231, 97200)
X_hog_test : (743, 97200)


In [9]:

# Color (H,S)
X_color_train = extract_color_hs_3x3(Training_origin_data, h_bins=45, s_bins=48)
X_color_test  = extract_color_hs_3x3(Test_data, h_bins=45, s_bins=48)

print("X_color_train:", X_color_train.shape)
print("X_color_test :", X_color_test.shape)


X_color_train: (2231, 837)
X_color_test : (743, 837)


In [21]:
from src.fpl_models import train_color_svm, train_hog_pca_svm_by_dims
from src.fpl_models import fit_sigmoid_calibrator, predict_proba_custom 
color_svm = train_color_svm(X_color_train, y_train_road, C=10, gamma="scale")

cal_color = fit_sigmoid_calibrator(color_svm, X_color_train, y_train_road,
                                  q_lo=0.10, q_hi=0.90, p_lo=0.05, p_hi=0.95) 
P_color_train = predict_proba_custom(color_svm, X_color_train,
                                    method="sigmoid", calibrator=cal_color, power=1.0)
P_color_test = predict_proba_custom(color_svm, X_color_test,
                                   method="sigmoid", calibrator=cal_color, power=1.0)  


print("P_color_test:", P_color_test.shape)

# 1) test 예측 라벨 (확률이 가장 큰 클래스)
y_pred_color = np.argmax(P_color_test, axis=1)

# 2) 정확도
acc_color = eval_accuracy(y_test_road, y_pred_color)
print("Color SVM Test Accuracy:", acc_color)

# 3) confusion matrix + report
cm_color = eval_confusion(y_test_road, y_pred_color)
print("\nColor SVM Confusion Matrix:\n", cm_color)

print("\nColor SVM Classification Report:\n")
print(eval_report(y_test_road, y_pred_color))


P_color_test: (743, 12)
Color SVM Test Accuracy: 0.721399730820996

Color SVM Confusion Matrix:
 [[51  1  0  0  6  0  0  0  1  2  1  0]
 [ 6 34  0  0  1  0  0  0  0  0  1  1]
 [ 2  0 39  2  2  2  2  1  1  1  0  6]
 [ 4  0  0 47  9  3  0  0  1  3  3  2]
 [10  1  4 10 60  2  0  0  4  3  5  1]
 [ 5  2  2  0  1 49  0  0  0  2  3  2]
 [ 2  1  1  0  0  0 21  1  0  0  0  0]
 [ 4  2  1  1  0  1  0 26  0  1  0  2]
 [ 2  2  0  0  5  0  0  1 46  3  0  0]
 [ 7  0  0  1  5  3  0  2  0 61  0  0]
 [ 5  0  0  0  5  2  0  0  1  3 27  0]
 [11  1  1  3  2  1  3  0  0  0  0 75]]

Color SVM Classification Report:

              precision    recall  f1-score   support

           0     0.4679    0.8226    0.5965        62
           1     0.7727    0.7907    0.7816        43
           2     0.8125    0.6724    0.7358        58
           3     0.7344    0.6528    0.6912        72
           4     0.6250    0.6000    0.6122       100
           5     0.7778    0.7424    0.7597        66
           6     0.8

In [22]:
PCA_DIMS = [2, 8, 16, 32, 64, 128, 256]

hog_pack = train_hog_pca_svm_by_dims(
    X_train=X_hog_train,
    y_train=y_train_road,
    X_test=X_hog_test,
    y_test=y_test_road,
    pca_dims=PCA_DIMS,
    C=10,
    gamma="scale"
)

# ✅ 1) 먼저 꺼내기
hog_svm_models = hog_pack["hog_svm_models"]
hog_pca_train_features = hog_pack["hog_pca_train_features"]
hog_pca_test_features  = hog_pack["hog_pca_test_features"]
hog_test_acc = hog_pack["test_acc"]
hog_time_report = hog_pack["time_report"]

# ✅ 2) calibrator 생성
cal_hog_by_dim = {}
for d in PCA_DIMS:
    cal_hog_by_dim[d] = fit_sigmoid_calibrator(
        hog_svm_models[d],
        hog_pca_train_features[d],
        y_train_road,
        q_lo=0.10, q_hi=0.90, p_lo=0.05, p_hi=0.95
    )

print("\n===== HOG+PCA SVM summary =====")
for d in PCA_DIMS:
    tr = hog_time_report[d]
    print(
        f"PCA {d:>3d} | "
        f"Test Acc={hog_test_acc[d]:.4f} | "
        f"scale={tr['scale_sec']:.3f}s pca={tr['pca_sec']:.3f}s svm={tr['svm_fit_sec']:.3f}s total={tr['total_sec']:.3f}s"
    )



===== HOG+PCA SVM summary =====
PCA   2 | Test Acc=0.2180 | scale=2.613s pca=1.402s svm=0.215s total=4.413s
PCA   8 | Test Acc=0.3526 | scale=2.691s pca=1.114s svm=0.153s total=4.139s
PCA  16 | Test Acc=0.5114 | scale=2.506s pca=1.164s svm=0.186s total=4.042s
PCA  32 | Test Acc=0.5424 | scale=2.284s pca=1.452s svm=0.195s total=4.133s
PCA  64 | Test Acc=0.5262 | scale=2.227s pca=2.116s svm=0.218s total=4.801s
PCA 128 | Test Acc=0.4899 | scale=2.325s pca=3.504s svm=0.247s total=6.424s
PCA 256 | Test Acc=0.4455 | scale=2.132s pca=3.950s svm=0.356s total=7.024s


ALPHA Line regression

In [26]:
from sklearn.linear_model import LogisticRegression

# =========================
# 입력 전제(이미 있어야 함)
# =========================
# PCA_DIMS: list[int]
# hog_svm_models: dict[dim -> SVC(probability=True)]
# hog_pca_test_features: dict[dim -> np.ndarray (N_test, dim)]
# P_color_test: np.ndarray (N_test, K)
# y_test_road: np.ndarray (N_test,)
# eval_accuracy: 함수(없으면 accuracy_score로 대체 가능)

fusion_lr_models = {}      # dim -> LogisticRegression
fusion_alpha = {}          # dim -> float (shape vs color 요약 alpha)
P_fusion_by_dim = {}       # dim -> np.ndarray (N_test, K)
fusion_acc = {}            # dim -> float

K = P_color_test.shape[1]

print("===== LR Fusion per PCA dim =====")

for d in PCA_DIMS:
    # 1) HOG 확률 (N_test, K)
    P_shape_test = predict_proba_custom(
    hog_svm_models[d],
    hog_pca_test_features[d],
    method="sigmoid",
    calibrator=cal_hog_by_dim[d],
    power=1.0
    )
    # shape 체크
    assert P_shape_test.shape == P_color_test.shape, (
        f"[dim={d}] P_shape_test {P_shape_test.shape} vs P_color_test {P_color_test.shape} mismatch"
    )

    # 2) fusion 입력 (N_test, 2K)
    X_fuse = np.hstack([P_shape_test, P_color_test])

    # 3) 회귀(로지스틱)로 fusion 학습 (요구대로 test 라벨 사용)
    lr = LogisticRegression(
    solver="lbfgs",
    max_iter=2000
    )

    lr.fit(X_fuse, y_test_road)

    # 4) fusion 확률 출력 (N_test, K)
    P_fusion_test = lr.predict_proba(X_fuse)

    # 5) 정확도(참고)
    y_pred = np.argmax(P_fusion_test, axis=1).astype(np.int64)
    acc = eval_accuracy(y_test_road, y_pred)

    # 6) "alpha(형태 비중)" 요약값 계산
    # lr.coef_ shape: (K, 2K)
    W = lr.coef_
    w_shape = np.mean(np.abs(W[:, :K]))
    w_color = np.mean(np.abs(W[:, K:]))

    alpha_hat = float(w_shape / (w_shape + w_color + 1e-12))  # 0~1, 형태 영향 비중

    # 저장
    fusion_lr_models[d] = lr
    fusion_alpha[d] = alpha_hat
    P_fusion_by_dim[d] = P_fusion_test
    fusion_acc[d] = float(acc)

    # 출력
    print(f"[PCA dim={d:>3d}] alpha_hat(shape)={alpha_hat:.3f} | TestAcc={acc:.4f} | P_fusion_test={P_fusion_test.shape}")

# dim별 요약
print("\n===== Summary =====")
for d in PCA_DIMS:
    print(f"dim={d:>3d} | alpha_hat={fusion_alpha[d]:.3f} | acc={fusion_acc[d]:.4f} | P_fusion={P_fusion_by_dim[d].shape}")

print("row-sum check:", P_color_test[0].sum(), P_shape_test[0].sum())



===== LR Fusion per PCA dim =====
[PCA dim=  2] alpha_hat(shape)=0.338 | TestAcc=0.7308 | P_fusion_test=(743, 12)
[PCA dim=  8] alpha_hat(shape)=0.399 | TestAcc=0.7322 | P_fusion_test=(743, 12)
[PCA dim= 16] alpha_hat(shape)=0.401 | TestAcc=0.7308 | P_fusion_test=(743, 12)
[PCA dim= 32] alpha_hat(shape)=0.418 | TestAcc=0.7429 | P_fusion_test=(743, 12)
[PCA dim= 64] alpha_hat(shape)=0.412 | TestAcc=0.7429 | P_fusion_test=(743, 12)
[PCA dim=128] alpha_hat(shape)=0.389 | TestAcc=0.7497 | P_fusion_test=(743, 12)
[PCA dim=256] alpha_hat(shape)=0.357 | TestAcc=0.7362 | P_fusion_test=(743, 12)

===== Summary =====
dim=  2 | alpha_hat=0.338 | acc=0.7308 | P_fusion=(743, 12)
dim=  8 | alpha_hat=0.399 | acc=0.7322 | P_fusion=(743, 12)
dim= 16 | alpha_hat=0.401 | acc=0.7308 | P_fusion=(743, 12)
dim= 32 | alpha_hat=0.418 | acc=0.7429 | P_fusion=(743, 12)
dim= 64 | alpha_hat=0.412 | acc=0.7429 | P_fusion=(743, 12)
dim=128 | alpha_hat=0.389 | acc=0.7497 | P_fusion=(743, 12)
dim=256 | alpha_hat=0.357

In [13]:
best_dim = max(fusion_acc, key=fusion_acc.get)

y_pred_best = np.argmax(P_fusion_by_dim[best_dim], axis=1)

cm = eval_confusion(y_test_road, y_pred_best)
print(f"Confusion Matrix (LR-Fusion, PCA={best_dim}):\n", cm)

print(f"\nClassification Report (LR-Fusion, PCA={best_dim}):\n")
print(eval_report(y_test_road, y_pred_best))


Confusion Matrix (LR-Fusion, PCA=128):
 [[49  1  0  0  7  1  0  0  1  2  1  0]
 [ 2 34  0  0  3  0  0  0  0  1  1  2]
 [ 1  0 39  2  2  2  0  1  1  1  0  9]
 [ 1  0  0 54  7  3  0  0  1  1  3  2]
 [ 1  1  5 11 65  2  0  0  4  4  5  2]
 [ 1  2  2  0  2 49  0  0  0  4  3  3]
 [ 0  1  1  2  0  0 20  1  0  0  0  1]
 [ 1  2  1  1  0  1  1 27  0  1  0  3]
 [ 0  2  0  0  6  0  0  1 47  3  0  0]
 [ 1  0  0  1  7  3  0  2  1 63  0  1]
 [ 1  0  0  0  7  2  0  1  1  3 27  1]
 [ 1  1  1  3  6  1  1  0  0  0  0 83]]

Classification Report (LR-Fusion, PCA=128):

              precision    recall  f1-score   support

           0     0.8305    0.7903    0.8099        62
           1     0.7727    0.7907    0.7816        43
           2     0.7959    0.6724    0.7290        58
           3     0.7297    0.7500    0.7397        72
           4     0.5804    0.6500    0.6132       100
           5     0.7656    0.7424    0.7538        66
           6     0.9091    0.7692    0.8333        26
           7

models save

In [20]:

import joblib

MODEL_DIR = "FPL_models"
os.makedirs(MODEL_DIR, exist_ok=True)

# =========================
# best_dim 기준 모델 선택
# =========================
best_dim = best_dim   # 이미 위에서 계산된 값 사용

hog_scaler = hog_pack["hog_pca_models"][best_dim][0]
hog_pca    = hog_pack["hog_pca_models"][best_dim][1]
hog_svm    = hog_pack["hog_svm_models"][best_dim]

# LR-fusion 모델 (있다면)
best_fusion_lr = fusion_lr_models[best_dim]
hog_cal = cal_hog_by_dim[best_dim]
color_cal = cal_color 
# =========================
# 모델 저장
# =========================
joblib.dump(hog_svm,        f"{MODEL_DIR}/hog_svm_dim{best_dim}.pkl")
joblib.dump(hog_pca,        f"{MODEL_DIR}/hog_pca_dim{best_dim}.pkl")
joblib.dump(hog_scaler,     f"{MODEL_DIR}/hog_scaler_dim{best_dim}.pkl")
joblib.dump(hog_cal,    f"{MODEL_DIR}/hog_cal_dim{best_dim}.pkl")
joblib.dump(color_cal, f"{MODEL_DIR}/color_cal.pkl")
joblib.dump(color_svm,      f"{MODEL_DIR}/color_svm.pkl")
joblib.dump(best_fusion_lr, f"{MODEL_DIR}/fusion_lr_dim{best_dim}.pkl")

joblib.dump(road_label_map, f"{MODEL_DIR}/road_label_map.pkl")

print("Models saved with best_dim =", best_dim)


Models saved with best_dim = 128


In [18]:
import importlib, inspect
import fpl_knn_models
import fpl_detail_models

print("loaded from:", fpl_knn_models.__file__)
importlib.reload(fpl_knn_models)
print("loaded from:", fpl_detail_models.__file__)
importlib.reload(fpl_detail_models)

print("new signature:", inspect.signature(fpl_knn_models.train_and_save_knn_models))

# ✅ 이 줄이 중요: 로컬 이름을 '새 함수'로 다시 바인딩
train_and_save_knn_models = fpl_knn_models.train_and_save_knn_models


loaded from: /home/hanseong/vscode/ML_code/FPL/FPL/src/fpl_knn_models.py
loaded from: /home/hanseong/vscode/ML_code/FPL/FPL/src/fpl_detail_models.py
new signature: (X_hog_train, X_color_train, training_road_label, training_detail, training_x, training_y, out_dir, hog_pca_dim=128, n_neighbors=7, detail_roads={'suporo_28', 'donhwamunro_11_ga', 'donhwamunro_11_na', 'donhwamunro_11_da'}, min_samples=10, training_paths=None, feature_tag='full')


In [19]:
MODEL_DIR = "FPL_models"  
ALPHA_FIXED = float(fusion_alpha[best_dim])   # 모든 디테일 도로에 동일 적용
PCA_DIM_DETAIL = best_dim                     # 디테일 HOG PCA도 동일 dim 사용(원하면 숫자 고정해도 됨)

training_paths = np.array([image_index.get(fn, "") for fn in training_filename], dtype=object)
# 1) detail 도로들에 대해 detail 분류용 (Color SVM + HOG SVM) 저장
detail_pack = train_and_save_detail_models(
    X_hog_train=X_hog_train,
    X_color_train=X_color_train,
    training_road_label=training_road_label,
    training_detail=training_detail,

    out_dir=MODEL_DIR,                
    alpha_shape=ALPHA_FIXED,          
    hog_pca_dim=PCA_DIM_DETAIL,       

    C=10,
    gamma="scale",

    min_total_samples=20,             
    min_samples_per_detail=8,
    feature_tag="patch3x3"          
)

detail_pack


# 2) 모든 도로명에 대해 KNN(위치 회귀) 저장 + detail 도로는 A/B/...별 KNN 추가 저장
train_and_save_knn_models(
    X_hog_train=X_hog_train,
    X_color_train=X_color_train,
    training_road_label=training_road_label,
    training_detail=training_detail,
    training_x=training_x,
    training_y=training_y,
    out_dir=MODEL_DIR,
    hog_pca_dim=best_dim,
    n_neighbors=7,
    min_samples=10,
    training_paths=training_paths,
    feature_tag="patch3x3"
)

[SAVED] detail(patch3x3) road=donhwamunro_11_da | classes=['A', 'B', 'C'] | n=177 | pca=128
[SAVED] detail(patch3x3) road=donhwamunro_11_ga | classes=['A', 'B', 'C'] | n=213 | pca=128
[SAVED] detail(patch3x3) road=donhwamunro_11_na | classes=['A', 'B', 'C', 'D'] | n=300 | pca=128
[SAVED] detail(patch3x3) road=suporo_28 | classes=['A', 'B', 'C', 'D', 'E'] | n=296 | pca=128
[SAVED] KNN(patch3x3) road=donhwamunro | n=189
[SAVED] KNN(patch3x3) road=donhwamunro_11 | n=130
[SAVED] KNN(patch3x3) road=donhwamunro_11_da | n=177
  [SAVED] KNN(patch3x3) road=donhwamunro_11_da detail=A | n=90
  [SAVED] KNN(patch3x3) road=donhwamunro_11_da detail=B | n=49
  [SAVED] KNN(patch3x3) road=donhwamunro_11_da detail=C | n=38
[SAVED] KNN(patch3x3) road=donhwamunro_11_ga | n=213
  [SAVED] KNN(patch3x3) road=donhwamunro_11_ga detail=A | n=46
  [SAVED] KNN(patch3x3) road=donhwamunro_11_ga detail=B | n=79
  [SAVED] KNN(patch3x3) road=donhwamunro_11_ga detail=C | n=88
[SAVED] KNN(patch3x3) road=donhwamunro_11_na

{'saved': [('donhwamunro', 'road'),
  ('donhwamunro_11', 'road'),
  ('donhwamunro_11_da', 'road'),
  ('donhwamunro_11_da', 'A'),
  ('donhwamunro_11_da', 'B'),
  ('donhwamunro_11_da', 'C'),
  ('donhwamunro_11_ga', 'road'),
  ('donhwamunro_11_ga', 'A'),
  ('donhwamunro_11_ga', 'B'),
  ('donhwamunro_11_ga', 'C'),
  ('donhwamunro_11_na', 'road'),
  ('donhwamunro_11_na', 'A'),
  ('donhwamunro_11_na', 'B'),
  ('donhwamunro_11_na', 'C'),
  ('donhwamunro_11_na', 'D'),
  ('samildaero', 'road'),
  ('samildaero_26', 'road'),
  ('samildaero_28', 'road'),
  ('samildaero_30', 'road'),
  ('samildaero_32', 'road'),
  ('samildaero_32_ga', 'road'),
  ('suporo_28', 'road'),
  ('suporo_28', 'A'),
  ('suporo_28', 'B'),
  ('suporo_28', 'C'),
  ('suporo_28', 'D'),
  ('suporo_28', 'E')],
 'knn_root': 'FPL_models/knn_models_patch3x3',
 'pca_dim_actual': 128,
 'feature_tag': 'patch3x3'}

In [127]:
# =========================
# KNN Evaluation (CORRECT version)
# =========================

from collections import defaultdict

MODEL_DIR = "FPL_models"
KNN_ROOT = os.path.join(MODEL_DIR, "knn_models_patch3x3")

# -------------------------
# 필수 변수 체크
# -------------------------
need = [
    "X_hog_test", "X_color_test",
    "y_test_road_label", "y_test_detail",
    "test_x", "test_y"
]
missing = [v for v in need if v not in globals()]
if missing:
    raise ValueError(f"❗ 필요한 변수가 없습니다: {missing}")

def _clean_detail(d):
    if d is None:
        return "0"
    s = str(d).strip()
    if s.lower() in ("nan", "none", ""):
        return "0"
    return s

def _mae_rmse(pred, true):
    pred = np.asarray(pred, dtype=float)
    true = np.asarray(true, dtype=float)
    diff = pred - true
    return float(np.mean(np.abs(diff))), float(np.sqrt(np.mean(diff**2)))

# -------------------------
# 1) 공통 HOG scaler / PCA 로드
# -------------------------
scaler_path = [p for p in os.listdir(KNN_ROOT) if p.startswith("knn_hog_scaler")][0]
pca_path    = [p for p in os.listdir(KNN_ROOT) if p.startswith("knn_hog_pca")][0]

hog_scaler = joblib.load(os.path.join(KNN_ROOT, scaler_path))
hog_pca    = joblib.load(os.path.join(KNN_ROOT, pca_path))

print("✅ loaded:", scaler_path, pca_path)

# -------------------------
# 2) test feature 변환 (🔥 핵심)
# -------------------------
Xh_test_p = hog_pca.transform(hog_scaler.transform(X_hog_test))
Z_test = np.hstack([Xh_test_p, X_color_test])

print("Z_test shape:", Z_test.shape)

# -------------------------
# 3) 예측
# -------------------------
N = len(Z_test)
pred_x = np.full(N, np.nan)
pred_y = np.full(N, np.nan)
missing_models = defaultdict(int)

y_test_road_label = np.asarray(y_test_road_label, dtype=object)
y_test_detail     = np.asarray([_clean_detail(d) for d in y_test_detail], dtype=object)

for i in range(N):
    road = str(y_test_road_label[i])
    det  = str(y_test_detail[i])

    # detail 우선 → road fallback
    cand = []
    if det != "0":
        cand.append(os.path.join(KNN_ROOT, road, f"knn_detail_{det}.pkl"))
    cand.append(os.path.join(KNN_ROOT, road, "knn_road.pkl"))

    model_path = None
    for p in cand:
        if os.path.exists(p):
            model_path = p
            break

    if model_path is None:
        missing_models[(road, det)] += 1
        continue

    knn = joblib.load(model_path)
    xy_hat = knn.predict(Z_test[i].reshape(1, -1)).ravel()
    pred_x[i], pred_y[i] = xy_hat

# -------------------------
# 4) 평가
# -------------------------
test_x = np.asarray(test_x, dtype=float)
test_y = np.asarray(test_y, dtype=float)

valid = np.isfinite(pred_x) & np.isfinite(pred_y)
idx = np.where(valid)[0]

if len(idx) == 0:
    print("❗ 예측 성공 샘플 없음")
    print("missing models:", dict(missing_models))
else:
    mae_x, rmse_x = _mae_rmse(pred_x[idx], test_x[idx])
    mae_y, rmse_y = _mae_rmse(pred_y[idx], test_y[idx])

    dist = np.sqrt((pred_x[idx] - test_x[idx])**2 + (pred_y[idx] - test_y[idx])**2)
    mae_dist  = float(np.mean(dist))
    rmse_dist = float(np.sqrt(np.mean(dist**2)))

    df = pd.DataFrame([{
        "n_eval": len(idx),
        "mae_x": mae_x,
        "rmse_x": rmse_x,
        "mae_y": mae_y,
        "rmse_y": rmse_y,
        "mae_dist": mae_dist,
        "rmse_dist": rmse_dist,
    }])

    print("\n✅ [KNN XY] Overall")
    display(df)

    # 도로별
    road_err = {}
    for r in np.unique(y_test_road_label[idx]):
        j = idx[y_test_road_label[idx] == r]
        road_err[str(r)] = float(np.mean(
            np.sqrt((pred_x[j] - test_x[j])**2 + (pred_y[j] - test_y[j])**2)
        ))

    df_road = pd.DataFrame([
        {"road": k, "mae_dist": v} for k, v in road_err.items()
    ]).sort_values("mae_dist")

    print("\n✅ Road-wise error")
    display(df_road)

    if missing_models:
        print("\n⚠️ Missing models (top-10):")
        for (road, det), cnt in sorted(missing_models.items(), key=lambda x: -x[1])[:10]:
            print(f"  road={road}, detail={det} → {cnt}")


✅ loaded: knn_hog_scaler_pca128.pkl knn_hog_pca_pca128.pkl
Z_test shape: (743, 965)

✅ [KNN XY] Overall


,n_eval,mae_x,rmse_x,mae_y,rmse_y,mae_dist,rmse_dist
0,743,0.466545,0.778131,0.484507,0.896316,0.788465,1.186959



✅ Road-wise error


,road,mae_dist
5,samildaero,0.000000e+00
9,samildaero_32,5.761906e-16
6,samildaero_26,1.995845e-01
7,samildaero_28,4.119368e-01
10,samildaero_32_ga,5.337569e-01
2,donhwamunro_11_da,8.596453e-01
1,donhwamunro_11,8.805383e-01
0,donhwamunro,9.174261e-01
11,suporo_28,1.055655e+00
8,samildaero_30,1.135010e+00


In [129]:
import psutil, os
print(f"RAM: {psutil.Process(os.getpid()).memory_info().rss / 1024**2:.1f} MB")


RAM: 20964.5 MB


In [130]:
# =========================
# 3x3 feature / model cleanup
# =========================
import gc

# ---- HOG / Color feature ----
for var in [
    "X_hog_train", "X_hog_test",
    "X_color_train", "X_color_test",
]:
    if var in globals():
        del globals()[var]

# ---- HOG PCA / SVM packs ----
for var in [
    "hog_pack",
    "hog_svm_models",
    "hog_pca_test_features",
    "hog_pca_train_features",
    "hog_test_acc",
    "hog_time_report",
]:
    if var in globals():
        del globals()[var]

# ---- Fusion 관련 ----
for var in [
    "P_color_test",
    "P_shape_test",
    "P_fusion_by_dim",
    "fusion_lr_models",
    "fusion_alpha",
    "fusion_acc",
]:
    if var in globals():
        del globals()[var]

# ---- Calibrators ----
for var in [
    "cal_color",
    "cal_hog_by_dim",
]:
    if var in globals():
        del globals()[var]

# ---- 임시 결과 ----
for var in [
    "y_pred_color",
    "y_pred_best",
    "cm",
]:
    if var in globals():
        del globals()[var]

# ---- Python GC ----
gc.collect()

print("✅ 3x3 variables cleaned. Ready for FULL feature extraction.")



✅ 3x3 variables cleaned. Ready for FULL feature extraction.


In [153]:
import psutil, os
print(f"RAM used: {psutil.Process(os.getpid()).memory_info().rss / 1024**2:.1f} MB")


RAM used: 19467.1 MB


not 3x3

In [154]:

from datetime import datetime
from sklearn.preprocessing import StandardScaler
from src.fpl_models import fit_sigmoid_calibrator, predict_proba_custom


# =========================
# 0) (이미 준비되어 있다고 가정)
# - Training_origin_data, Test_data
# - y_train_road, y_test_road
# - road_label_map  (있으면 저장용)
# =========================

# =========================
# 1) Feature Extraction (FULL)
# =========================
# Color (HS hist) - full image (차원 고정이라 안전)
X_color_train = extract_color_hs_full(
    Training_origin_data, h_bins=60, s_bins=64, sizes=None
)
X_color_test  = extract_color_hs_full(
    Test_data,            h_bins=60, s_bins=64, sizes=None
)

# HOG - full image (반드시 리사이즈해서 차원 고정)
X_hog_train = extract_hog_full(
    Training_origin_data,
    hog_sizes=((256, 128),),      
    orientations=12,
    pixels_per_cell=(8, 8),
    cells_per_block=(2, 2),
)
X_hog_test = extract_hog_full(
    Test_data,
    hog_sizes=((256, 128),),
    orientations=12,
    pixels_per_cell=(8, 8),
    cells_per_block=(2, 2),
)

print("X_color_train:", X_color_train.shape, X_color_train.dtype)
print("X_hog_train  :", X_hog_train.shape,   X_hog_train.dtype)


X_color_train: (2231, 124) float32
X_hog_train  : (2231, 22320) float32


In [155]:

# =========================
# 2) Scaling
# =========================
sc_color = StandardScaler()
Xc_tr = sc_color.fit_transform(X_color_train)
Xc_te = sc_color.transform(X_color_test)

# HOG는 PCA+SVM 함수 안에서 scaler를 따로 쓰는 구조가 많아서
# 여기서는 "raw HOG"를 그대로 넘김 (너가 쓰던 방식 유지)
Xh_tr, Xh_te = X_hog_train, X_hog_test


In [156]:

# =========================
# 3) Train COLOR SVM + Evaluate
# =========================
svm_color = train_color_svm(Xc_tr, y_train_road, C=10, gamma="scale")
res_color = eval_svm(svm_color, Xc_tr, y_train_road, Xc_te, y_test_road, name="COLOR_SVM")

print("\n[COLOR] result:", res_color)



[COLOR] result: {'name': 'COLOR_SVM', 'train_acc': 0.9627969520394442, 'test_acc': 0.7012113055181696, 'pred_sec': 0.31446190499991644}


In [157]:

# =========================
# 4) Train HOG PCA+SVM by dims + pick best
# =========================
pca_dims = [32, 64, 128, 256]   # 필요하면 조절
hog_result = train_hog_pca_svm_by_dims(
    Xh_tr, y_train_road,
    Xh_te, y_test_road,
    pca_dims=pca_dims,
    C=10,
    gamma="scale",
)

df_hog = pd.DataFrame({
    "dim": pca_dims,
    "train_acc": [hog_result["train_acc"][d] for d in pca_dims],
    "test_acc":  [hog_result["test_acc"][d]  for d in pca_dims],
    "total_sec": [hog_result["time_report"][d]["total_sec"] for d in pca_dims],
}).sort_values("test_acc", ascending=False)

print("\n[HOG PCA+SVM] leaderboard\n", df_hog)

best_dim = int(df_hog.iloc[0]["dim"])
print("\n✅ best_dim =", best_dim)

# best 모델 꺼내기 (너가 쓰던 pack 구조 기준)
hog_pack = hog_result["pack"] if "pack" in hog_result else hog_result  # 방어코드
hog_scaler = hog_pack["hog_pca_models"][best_dim][0]
hog_pca    = hog_pack["hog_pca_models"][best_dim][1]
svm_hog    = hog_pack["hog_svm_models"][best_dim]



[HOG PCA+SVM] leaderboard
    dim  train_acc  test_acc  total_sec
1   64   1.000000  0.504711   1.694379
0   32   0.999552  0.499327   1.832274
2  128   1.000000  0.484522   2.014503
3  256   1.000000  0.422611   2.522735

✅ best_dim = 64


In [158]:
# =========================
# 5) Calibration -> probability (COLOR/HOG)  ✅ custom
# =========================
# (1) HOG는 PCA된 feature로 확률을 만들어야 함
Xh_tr_p = hog_pca.transform(hog_scaler.transform(Xh_tr))
Xh_te_p = hog_pca.transform(hog_scaler.transform(Xh_te))

# (2) calibrator는 "train feature"로 fit
cal_color = fit_sigmoid_calibrator(
    svm_color, Xc_tr, y_train_road,
    q_lo=0.10, q_hi=0.90, p_lo=0.05, p_hi=0.95
)
cal_hog = fit_sigmoid_calibrator(
    svm_hog, Xh_tr_p, y_train_road,
    q_lo=0.10, q_hi=0.90, p_lo=0.05, p_hi=0.95
)

# (3) test에서 custom 확률 생성
P_color = predict_proba_custom(
    svm_color, Xc_te,
    method="sigmoid", calibrator=cal_color, power=1.0
)
P_hog = predict_proba_custom(
    svm_hog, Xh_te_p,
    method="sigmoid", calibrator=cal_hog, power=1.0
)

print("row-sum check:", float(P_color[0].sum()), float(P_hog[0].sum()))


row-sum check: 1.0 1.0


In [159]:
# --- train 확률도 만들어야 LR-fusion 학습 가능 ---
P_color_tr = predict_proba_custom(svm_color, Xc_tr,  method="sigmoid", calibrator=cal_color, power=1.0)
P_hog_tr   = predict_proba_custom(svm_hog,   Xh_tr_p, method="sigmoid", calibrator=cal_hog,   power=1.0)


In [166]:
from sklearn.linear_model import LogisticRegression

# =========================
# 6) Fusion (LR) + alpha_hat
# =========================
K = P_color.shape[1]
assert P_hog.shape == P_color.shape, "P_hog / P_color shape mismatch"

# (1) LR 학습용 입력: (N_train, 2K)
X_fuse_tr = np.hstack([P_hog_tr, P_color_tr])

# (2) LR 학습 (권장: train 라벨로 fit)
lr_fusion = LogisticRegression(solver="lbfgs", max_iter=2000)
lr_fusion.fit(X_fuse_tr, y_train_road)

# (3) test에서 fusion 확률
X_fuse_te = np.hstack([P_hog, P_color])
P_final = lr_fusion.predict_proba(X_fuse_te)

# (4) 성능
final_acc, final_pred = evaluate_fusion(P_final, y_test_road)
acc_hog,   _ = evaluate_fusion(P_hog,   y_test_road)
acc_color, _ = evaluate_fusion(P_color, y_test_road)

# (5) alpha_hat 추정 (shape 영향 비중)
W = lr_fusion.coef_               # (K, 2K)
w_shape = float(np.mean(np.abs(W[:, :K])))
w_color = float(np.mean(np.abs(W[:, K:])))
alpha_hat = float(w_shape / (w_shape + w_color + 1e-12))

print(f"✅ alpha_hat(shape) = {alpha_hat:.3f}")
print("SCORES:", {"HOG": acc_hog, "COLOR": acc_color, "FUSION(LR)": final_acc})


✅ alpha_hat(shape) = 0.618
SCORES: {'HOG': 0.46837146702557203, 'COLOR': 0.6783310901749664, 'FUSION(LR)': 0.5531628532974427}


In [161]:

# =========================
# 7) Save models (FULL: color+hog only)
# =========================
MODEL_DIR = "FPL_models"
os.makedirs(MODEL_DIR, exist_ok=True)

ts = datetime.now().strftime("%Y%m%d_%H%M%S")
tag = f"FULL{best_dim}"

# ---- road models ----
joblib.dump(svm_hog,    f"{MODEL_DIR}/hog_svm_{tag}.pkl")
joblib.dump(hog_pca,    f"{MODEL_DIR}/hog_pca_{tag}.pkl")
joblib.dump(hog_scaler, f"{MODEL_DIR}/hog_scaler_{tag}.pkl")

joblib.dump(svm_color,  f"{MODEL_DIR}/color_svm_{tag}.pkl")
joblib.dump(sc_color,   f"{MODEL_DIR}/color_scaler_{tag}.pkl")

# ---- calibrators ----
joblib.dump(cal_hog,    f"{MODEL_DIR}/cal_hog_{tag}.pkl")
joblib.dump(cal_color,  f"{MODEL_DIR}/cal_color_{tag}.pkl")


# ---- label map (있으면) ----
if "road_label_map" in globals():
    joblib.dump(road_label_map, f"{MODEL_DIR}/road_label_map_{tag}.pkl")


joblib.dump(lr_fusion, f"{MODEL_DIR}/fusion_lr_{tag}.pkl")
joblib.dump({"alpha_hat": alpha_hat}, f"{MODEL_DIR}/fusion_alpha_{tag}.pkl")

# ---- meta ----
joblib.dump({
    "hog_resize": (256, 256),
    "hog_orientations": 12,
    "hog_pixels_per_cell": (8, 8),
    "hog_cells_per_block": (2, 2),
    "color_h_bins": 60,
    "color_s_bins": 64,
    "best_dim": best_dim,
}, f"{MODEL_DIR}/meta_{tag}.pkl")

print("\n✅ Models saved")
print("Dir =", MODEL_DIR)
print("Tag =", tag)



✅ Models saved
Dir = FPL_models
Tag = FULL64


detail_SVM and kNN

In [162]:
import importlib, inspect
import fpl_knn_models
import fpl_detail_models

print("loaded from:", fpl_knn_models.__file__)
importlib.reload(fpl_knn_models)
print("loaded from:", fpl_detail_models.__file__)
importlib.reload(fpl_detail_models)

print("new signature:", inspect.signature(fpl_knn_models.train_and_save_knn_models))

# ✅ 이 줄이 중요: 로컬 이름을 '새 함수'로 다시 바인딩
train_and_save_knn_models = fpl_knn_models.train_and_save_knn_models


loaded from: /home/hanseong/vscode/ML_code/FPL/FPL/src/fpl_knn_models.py
loaded from: /home/hanseong/vscode/ML_code/FPL/FPL/src/fpl_detail_models.py
new signature: (X_hog_train, X_color_train, training_road_label, training_detail, training_x, training_y, out_dir, hog_pca_dim=128, n_neighbors=7, detail_roads={'donhwamunro_11_ga', 'donhwamunro_11_na', 'donhwamunro_11_da', 'suporo_28'}, min_samples=10, training_paths=None, feature_tag='full')


In [163]:
# ---- reload ----
import importlib, inspect
import fpl_knn_models
import fpl_detail_models

importlib.reload(fpl_knn_models)
importlib.reload(fpl_detail_models)

train_and_save_knn_models = fpl_knn_models.train_and_save_knn_models

MODEL_DIR = "FPL_models"
feature_tag = tag                
ALPHA_FIXED = float(alpha_hat)       


detail_pack = train_and_save_detail_models(
    X_hog_train=Xh_tr_p,         
    X_color_train=Xc_tr,         
    training_road_label=training_road_label,
    training_detail=training_detail,
    out_dir=MODEL_DIR,
    alpha_shape=ALPHA_FIXED,
    hog_pca_dim=best_dim,        
    C=10, gamma="scale",
    min_total_samples=20,
    min_samples_per_detail=8,
    feature_tag=feature_tag,
)


knn_pack = train_and_save_knn_models(
    X_hog_train=X_hog_train,
    X_color_train=Xc_tr,
    training_road_label=training_road_label,
    training_detail=training_detail,
    training_x=training_x,
    training_y=training_y,
    out_dir=MODEL_DIR,
    hog_pca_dim=best_dim,        
    n_neighbors=10,
    min_samples=10,
    training_paths=training_paths,
    feature_tag=feature_tag,
)


[SAVED] detail(FULL64) road=donhwamunro_11_da | classes=['A', 'B', 'C'] | n=177 | pca=64
[SAVED] detail(FULL64) road=donhwamunro_11_ga | classes=['A', 'B', 'C'] | n=213 | pca=64
[SAVED] detail(FULL64) road=donhwamunro_11_na | classes=['A', 'B', 'C', 'D'] | n=300 | pca=64
[SAVED] detail(FULL64) road=suporo_28 | classes=['A', 'B', 'C', 'D', 'E'] | n=296 | pca=64
[SAVED] KNN(FULL64) road=donhwamunro | n=189
[SAVED] KNN(FULL64) road=donhwamunro_11 | n=130
[SAVED] KNN(FULL64) road=donhwamunro_11_da | n=177
  [SAVED] KNN(FULL64) road=donhwamunro_11_da detail=A | n=90
  [SAVED] KNN(FULL64) road=donhwamunro_11_da detail=B | n=49
  [SAVED] KNN(FULL64) road=donhwamunro_11_da detail=C | n=38
[SAVED] KNN(FULL64) road=donhwamunro_11_ga | n=213
  [SAVED] KNN(FULL64) road=donhwamunro_11_ga detail=A | n=46
  [SAVED] KNN(FULL64) road=donhwamunro_11_ga detail=B | n=79
  [SAVED] KNN(FULL64) road=donhwamunro_11_ga detail=C | n=88
[SAVED] KNN(FULL64) road=donhwamunro_11_na | n=300
  [SAVED] KNN(FULL64) roa

In [164]:
import os, json, joblib
import numpy as np
import pandas as pd
import gc

# -------------------------
# 0) 유틸
# -------------------------
def clean_detail_arr(detail_arr):
    out = []
    for d in detail_arr:
        if d is None:
            out.append("0"); continue
        s = str(d).strip()
        if s.lower() in ("nan", "none", ""):
            out.append("0")
        else:
            out.append(s)
    return np.array(out, dtype=object)

def knn_root_from_tag(model_dir, feature_tag):
    ft = (feature_tag or "").strip()
    if ft in ("", "legacy"):
        return os.path.join(model_dir, "knn_models")
    return os.path.join(model_dir, f"knn_models_{ft}")

def load_knn_global_scaler_pca(knn_root, pca_dim):
    scaler_path = os.path.join(knn_root, f"knn_hog_scaler_pca{pca_dim}.pkl")
    pca_path    = os.path.join(knn_root, f"knn_hog_pca_pca{pca_dim}.pkl")
    if not (os.path.exists(scaler_path) and os.path.exists(pca_path)):
        raise FileNotFoundError(f"❗ scaler/pca not found:\n{scaler_path}\n{pca_path}")
    scaler = joblib.load(scaler_path)
    pca    = joblib.load(pca_path)
    print("✅ loaded:", os.path.basename(scaler_path), os.path.basename(pca_path))
    return scaler, pca

def build_Z(X_hog, X_color, scaler, pca):
    Xh_p = pca.transform(scaler.transform(X_hog))
    return np.hstack([Xh_p, X_color])

# -------------------------
# 1) 경로/로드
# -------------------------
KNN_PCA_DIM = int(best_dim)  # 너 코드에서 best_dim 그대로 사용
knn_root = knn_root_from_tag(MODEL_DIR, feature_tag)
print("knn_root =", knn_root)

hog_scaler_knn, hog_pca_knn = load_knn_global_scaler_pca(knn_root, KNN_PCA_DIM)

# -------------------------
# 2) test feature 변환 (🔥 KNN은 'KNN용 scaler/pca'로 변환해야 함)
# -------------------------
# ⚠️ 주의: X_hog_test는 "raw HOG" 여야 함. (지금 셀에서는 full raw hog로 추출한 상태)
#         만약 X_hog_test가 이미 PCA된 걸 넣으면 차원 깨져서 에러남.
Z_test = build_Z(X_hog_test, X_color_test, hog_scaler_knn, hog_pca_knn)
print("Z_test shape:", Z_test.shape)

# -------------------------
# 3) 도로별 KNN 예측 (road-level)
# -------------------------
road_arr  = np.asarray(test_road_label, dtype=object)   # 문자열 도로명
dt_arr    = clean_detail_arr(test_detail)               # detail 문자열 (없으면 "0")

pred_x = np.full(len(road_arr), np.nan, dtype=np.float32)
pred_y = np.full(len(road_arr), np.nan, dtype=np.float32)
ok = np.zeros(len(road_arr), dtype=bool)

missing_models = {}

for i in range(len(road_arr)):
    road = road_arr[i]
    model_path = os.path.join(knn_root, road, "knn_road.pkl")

    if not os.path.exists(model_path):
        key = (road, "road")
        missing_models[key] = missing_models.get(key, 0) + 1
        continue

    knn = joblib.load(model_path)
    xy_hat = knn.predict(Z_test[i].reshape(1, -1)).ravel()
    pred_x[i], pred_y[i] = float(xy_hat[0]), float(xy_hat[1])
    ok[i] = True

print("ok preds:", int(ok.sum()), "/", len(ok))

if ok.sum() == 0:
    print("❗ 예측 성공한 샘플이 없어서 평가 불가.")
    print("missing model examples:", sorted(missing_models.items(), key=lambda x: -x[1])[:10])
else:
    # -------------------------
    # 4) 평가 (test_x/test_y 있을 때)
    # -------------------------
    if ("test_x" in globals()) and ("test_y" in globals()):
        gt_x = np.asarray(test_x, dtype=np.float32)
        gt_y = np.asarray(test_y, dtype=np.float32)

        # 유효한 gt + 예측 성공만
        valid = ok & np.isfinite(gt_x) & np.isfinite(gt_y)
        n_eval = int(valid.sum())

        if n_eval == 0:
            print("⚠️ test_x/test_y가 있지만 유효한 좌표가 없어서 스킵")
        else:
            dx = pred_x[valid] - gt_x[valid]
            dy = pred_y[valid] - gt_y[valid]
            dist = np.sqrt(dx*dx + dy*dy)

            mae_x = float(np.mean(np.abs(dx)))
            rmse_x = float(np.sqrt(np.mean(dx*dx)))
            mae_y = float(np.mean(np.abs(dy)))
            rmse_y = float(np.sqrt(np.mean(dy*dy)))
            mae_dist = float(np.mean(dist))
            rmse_dist = float(np.sqrt(np.mean(dist*dist)))

            overall = pd.DataFrame([{
                "n_eval": n_eval,
                "mae_x": mae_x, "rmse_x": rmse_x,
                "mae_y": mae_y, "rmse_y": rmse_y,
                "mae_dist": mae_dist, "rmse_dist": rmse_dist,
            }])
            display(overall)

            # road-wise
            rows = []
            for r in sorted(set(road_arr[valid])):
                idx = valid & (road_arr == r)
                if idx.sum() == 0: 
                    continue
                dxr = pred_x[idx] - gt_x[idx]
                dyr = pred_y[idx] - gt_y[idx]
                dist_r = np.sqrt(dxr*dxr + dyr*dyr)
                rows.append({"road": r, "mae_dist": float(np.mean(dist_r))})

            roadwise = pd.DataFrame(rows).sort_values("mae_dist")
            display(roadwise)
    else:
        print("⚠️ test_x/test_y가 없어서 '예측 생성'까지만 완료 (평가 지표는 스킵)")


knn_root = FPL_models/knn_models_FULL64
✅ loaded: knn_hog_scaler_pca64.pkl knn_hog_pca_pca64.pkl
Z_test shape: (743, 188)
ok preds: 743 / 743


,n_eval,mae_x,rmse_x,mae_y,rmse_y,mae_dist,rmse_dist
0,743,0.535089,0.774125,0.53916,0.892549,0.883208,1.181488


,road,mae_dist
5,samildaero,0.000000
9,samildaero_32,0.000000
6,samildaero_26,0.182142
7,samildaero_28,0.482632
10,samildaero_32_ga,0.707347
1,donhwamunro_11,0.906444
2,donhwamunro_11_da,1.044528
0,donhwamunro,1.068730
11,suporo_28,1.120410
4,donhwamunro_11_na,1.326793


In [152]:
import gc

# -------------------------
# FULL pipeline 대용량 변수 정리
# -------------------------
to_del = [
    # raw/full features
    "X_color_train", "X_color_test",
    "X_hog_train", "X_hog_test",
    "Xc_tr", "Xc_te",
    "Xh_tr", "Xh_te",
    "Xh_tr_p", "Xh_te_p",

    # probabilities / fusion
    "P_color", "P_hog", "P_color_tr", "P_hog_tr",
    "X_fuse_tr", "X_fuse_te", "P_final",
    "final_pred",

    # models/calibrators
    "svm_color", "svm_hog",
    "hog_scaler", "hog_pca",
    "cal_color", "cal_hog",
    "lr_fusion",

    # hog search packs
    "hog_result", "hog_pack", "df_hog",

    # detail/knn packs
    "detail_pack", "knn_pack",

    # KNN eval intermediates
    "hog_scaler_knn", "hog_pca_knn",
    "Z_test", "pred_x", "pred_y", "ok",
    "missing_models",
]

for v in to_del:
    if v in globals():
        del globals()[v]

gc.collect()
print("✅ Cleanup done: freed FULL/knn intermediate variables.")


✅ Cleanup done: freed FULL/knn intermediate variables.
